# Big Box simulation computational cost estimator

Use with caution.  c_s and v_s are educated estimates.  Note that this assumes that the time steps are really constrained by the c_s and v_s given; the real average time step is somewhat lower than this, but my goal here is to actually give sensible limits so we don't ask for too little computational time. 

In [1]:
Lbox = 25.0e+3 / 0.7 # 25 Mpc in kpc (the / 0.7 cleans out the factor of h)
Nbox = 2048  # root grid num cells
Max_level = 4  # maximum level of refinement

# c_s and v_s are sound speed and peculiar gas speed in typical large halos in the box
c_s = 100.0 * 1.e5  # 100 km/s in cm/s
v_s = 300.0 * 1.e5  #  

# assume that this cubed is the number of root grid cells per compute core (32 to 64 is sensible)
cells_core = 64

# this is Enzo's approximate speed on Pleiades with All The Physics, and thus we assume Enzo-E's
cell_updates_core_second = 5.0e+3

# at what redshift do you want this simulation to stop?
simulation_end_redshift = 0.0

# we assume that there are the same number of cells on each level (lagrangian-like refinement);
# this multiplier increases that if we assume we have more aggressive refinement criteria
cell_safety_factor = 2  

# number of data outputs (required for data volume estimate)
number_of_data_outputs = 100

print("Root grid dx:  {:.3f} com. kpc ".format(Lbox/Nbox))
print("Max level dx:  {:.3f} com. kpc (L_max = {:d})".format(Lbox/Nbox/2**Max_level, Max_level))


Root grid dx:  17.439 com. kpc 
Max level dx:  1.090 com. kpc (L_max = 4)


In [2]:
from astropy.cosmology import WMAP9 as cosmo

simulation_end_time = (cosmo.age(simulation_end_redshift)).value  # simulation end time in gyr 

print("simulation end time {:.2f} Gyr".format(simulation_end_time))

/Users/molly/anaconda3/envs/astroconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


simulation end time 13.77 Gyr


In [3]:
# some useful conversions

Lbox_cm = Lbox * 3.08e21 # box in cm
seconds_per_yr = 365.25*86400
seconds_per_myr = 1.0e6 * seconds_per_yr
universe_age = 13.8e9 * seconds_per_yr

simulation_end_time_seconds = simulation_end_time * 1.0e9 * seconds_per_yr # simulation end time in gyr

In [4]:
# total cells in root grid
total_cells_RG = Nbox**3

dx_rg = Lbox_cm / Nbox     # root grid dx (in cm)

dt_rg = 0.2 * dx_rg / (c_s + v_s)   # estimate of root grid timestep in seconds (hydro courant time)

dx_max = dx_rg / 2**Max_level  # cell size at max level in cm

dt_max = dt_rg / 2**Max_level  # timestep at max level in seconds

# number of root grid timesteps
num_rg_ts = simulation_end_time_seconds / dt_rg

# number of max level timesteps (should be much larger than the root grid)
num_maxlev_ts = simulation_end_time_seconds / dt_max

print("Root grid timestep {:.2e} Myrs".format(dt_rg/seconds_per_myr))
print("Max level timestep {:.2e} Myrs (L_max = {:d})".format(dt_max/seconds_per_myr,Max_level))

print("num RG timesteps per simulation: {:.2f}".format(num_rg_ts))
print("num max level timesteps per simulation: {:.2f}".format(num_maxlev_ts))

Root grid timestep 8.51e+00 Myrs
Max level timestep 5.32e-01 Myrs (L_max = 4)
num RG timesteps per simulation: 1617.97
num max level timesteps per simulation: 25887.52


## Conservative estimate

This is extremely conservative, and represents a high upper bound to the estimated simulation runtime.  It assumes universal time steps for the simulation - namely, all time steps are at the timestep of the max level of refinement.  **This represents the worst-case scenario**, if Enzo-E's local causality-preserving time steps are not ready by the time this simulation is run.

In [5]:
# total number of cells in simulation
total_cells = total_cells_RG * (1+Max_level) * cell_safety_factor

# total cell updates per max level timestep (which will dominate simulation time)
total_cell_updates = total_cells * num_maxlev_ts

# total core hours: cell updates / (cell updates per core-hour)
total_core_hours = total_cell_updates / (cell_updates_core_second * 3600.0)

print("{:e} total cells".format(total_cells))

print("{:e} total cell updates".format(total_cell_updates))

print("{:e} total core hours".format(total_core_hours))

cores = (Nbox//cells_core)**3

print("number of cores needed:", cores, "at", str(cells_core)+"^3 root grid cells/core")
print("total wall clock hours:  {:.2f}".format(total_core_hours/cores))

8.589935e+10 total cells
2.223721e+15 total cell updates
1.235401e+08 total core hours
number of cores needed: 32768 at 64^3 root grid cells/core
total wall clock hours:  3770.14


## Optimistic estimate

This is an estimate that should be taken as a lower bound to the estimated simulation runtime.  It assumes that each grid takes its own locally-determined time step.  **This represents the best-case scenario,** assuming Enzo-E's local causality-preserving timesteps are READY by the time the simulation is run.

In [6]:
total_cell_updates = 0

# total cell updates per root grid timestep (grids at level L+1 take time steps that
# are half the length of those on level L, on average)
for i in range(Max_level+1):
    total_cell_updates += total_cells_RG * 2.0**i
    #print("level", i, "time steps", 2.0**i, "total updates", total_cell_updates)

# multiply in cell safety factor (taking into account ghost zones, etc.)
total_cell_updates *= cell_safety_factor

# times number of root grid time steps to figure out total number of cell updates
total_cell_updates *= num_rg_ts 

# total core hours: cell updates / (cell updates per core-hour)
total_core_hours = total_cell_updates / (cell_updates_core_second * 3600.0)

print("{:e} total cells".format(total_cells))

print("{:e} total cell updates per".format(total_cell_updates))

print("{:e} total core hours".format(total_core_hours))

cores = (Nbox//cells_core)**3

print("number of cores needed:", cores, "at", str(cells_core)+"^3 root grid cells/core")
print("total wall clock hours:  {:.2f}".format(total_core_hours/cores))

8.589935e+10 total cells
8.616919e+14 total cell updates per
4.787177e+07 total core hours
number of cores needed: 32768 at 64^3 root grid cells/core
total wall clock hours:  1460.93


# Data volume estimation

In [7]:
bytes_per_float = 8 # double precision

# density, total energy, internal energy, 3*velocity, 3*mag field, 6 metal species + "metal",
# 6 primordial species (atomic H, He, and e-), grav. potential
num_fields = 24 

# total_cells includes all levels _ safety factor
data_volume_per_output = total_cells * bytes_per_float * num_fields 

print("each output is: {:.3f} TB".format(data_volume_per_output/1.e12))

# scales data volume up by expected number of data outputs
data_volume_total = data_volume_per_output*number_of_data_outputs

print("total raw data volume is: {:.3f} TB".format(data_volume_total/1.e12))

print("compressed data volume is:  {:.3f} TB".format(data_volume_total/1.e12/5))

each output is: 16.493 TB
total raw data volume is: 1649.267 TB
compressed data volume is:  329.853 TB
